## Example - 0D hydrogen CRM using CRM modelbound data and term generator functionality with kinetic electrons

This is the v2.0.0 rewrite of the notebook that corresponds to the kinetic test in Section 5.3. of the ReMKiT1D code paper. It tests particle and energy conservation properties of SOL-KiT-like implementations of electron-neutral Boltzmann collisions from common_models.py

In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.crm_support as crm # This is the main python module for crm construction
from RMK_support import node,varFromNode

### Some useful constants

In [ ]:
heavySpeciesMass = 2.014 #in amus
hPlanck = 6.62607004e-34
elMass =  9.10938e-31
elCharge = 1.60218e-19


### Context initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_kin_crm_test/")
rk.normTemperature = 5.0

### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D
dv0 = 0.01
cv = 1.025
vGrid = [dv0]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 0 
rk.grid = rmk.Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True ,interpretVGridAsWidths=True)

### Species initialization 

NOTE: The CRM density evolution generator assumes that all species it evolves have a density as their first associated variable and that it is an implicit and fluid variable. Also, the prebuilt Janev data requires that neutral IDs correspond to principle quantum numbers of excited states.

In [ ]:
electronSpecies = rmk.Species("e",0) 
ionSpecies = rmk.Species("D+",-1)

rk.species.add(electronSpecies,ionSpecies)

numNeutrals=20
neutralDensList = ["n"+str(i) for i in range(1,numNeutrals+1)] # List of neutral density names
neutralSpecies = []
for neutral in neutralDensList:
    neutralSpecies.append(rmk.Species("D"+neutral[1:],int(neutral[1:]),heavySpeciesMass))
rk.species.add(*tuple(neutralSpecies))

### Variable initialization

In [ ]:
TInit = np.ones(rk.grid.numX)

nInit = np.ones(rk.grid.numX)  
n1Init = 0.1*np.ones(rk.grid.numX)

fInit = np.zeros([rk.grid.numX,rk.grid.numH,rk.grid.numV])
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = (TInit[i]*np.pi)**(-1.5) * nInit[i] * np.exp(-rk.grid.vGrid**2/TInit[i])

# Rescale distribution function to ensure that the numerical density moment agrees with the initial values
numerical_dens = rk.grid.velocityMoment(fInit,0,1)
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = nInit[i] *fInit[i,rk.grid.getH(0)-1,:]/numerical_dens[i]

f = rmk.Variable("f",rk.grid,data=fInit,isDistribution=True)
We = rmk.Variable("We",rk.grid,derivation=rk.textbook["energyMoment"],derivationArgs=["f"])
ne = rmk.Variable("ne",rk.grid,derivation=rk.textbook["densityMoment"],derivationArgs=["f"])
electronSpecies.associateVar(ne)
Te =  varFromNode("Te",rk.grid,node=(2/3)*node(We)/node(ne))

rk.variables.add(f,We,ne,Te)
ni = rmk.Variable("ni",rk.grid,data=nInit,units='$10^{19} m^{-3}$')
ionSpecies.associateVar(ni)
rk.variables.add(ni)

neutralVars = []

neutralVars.append(rmk.Variable("n1",rk.grid,data=n1Init*np.ones(rk.grid.numX),units='$10^{19} m^{-3}$'))
for i in range(2,numNeutrals+1):
    neutralVars.append(rmk.Variable(neutralDensList[i-1],rk.grid,units='$10^{19} m^{-3}$'))

for i,var in enumerate(neutralVars):
    neutralSpecies[i].associateVar(var)
rk.variables.add(*tuple(neutralVars))

#### Generating modelbound data based on inbuilt cross-sections

In [ ]:
includedJanevTransitions = ["ex","deex","ion","recomb3b"]
mbData = crm.CRMModelboundData(rk.grid)
crm.addJanevTransitionsToCRMData(mbData,numNeutrals,rk.norms["eVTemperature"],f,Te,detailedBalanceCSPriority=1,processes=includedJanevTransitions,lowestCellEnergy=rk.grid.vGrid[0]**2)

#### Creating the CRM model object

In [ ]:
#CRM model

#Initializing model
crmModel = rmk.Model("CRM")

crmModel.setModelboundData(mbData)

#Add term generator responsible for buildling CRM model for ions

crmModel.addTermGenerator(crm.CRMTermGenerator("ionCRM",evolvedSpecies=[ionSpecies]))

#Add term generator responsible for buildling CRM model for all neutral states
crmModel.addTermGenerator(crm.CRMTermGenerator("neutralCRM",neutralSpecies))

#Add Boltzmann term generator for excitation
exInds = mbData.getTransitionIndices("JanevEx")

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("exCRME",f,1,exInds,mbData))

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("exCRMA",f,1,exInds,mbData,absorptionTerms=True))

#Add Boltzmann term generators for ionization

ionInds = mbData.getTransitionIndices("JanevIon")

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("ionCRME",f,1,ionInds,mbData))

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("ionCRMA",f,1,ionInds,mbData,absorptionTerms=True))

#Add Boltzmann term generators for deexcitation

deexInds = mbData.getTransitionIndices("JanevDeex")

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("deexCRME",f,1,deexInds,mbData))

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("deexCRMA",f,1,deexInds,mbData,absorptionTerms=True))

# #Add Boltzmann term generators for 3b recombination

recomb3bInds = mbData.getTransitionIndices("JanevRecomb3b")

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("recomb3bCRME",f,1,recomb3bInds,mbData))

crmModel.addTermGenerator(crm.CRMBoltzTermGenerator("recomb3bCRMA",f,1,recomb3bInds,mbData,absorptionTerms=True))

#Add secondary electron sources/sinks due to ionization and recombination

secElInds = ionInds + recomb3bInds 

crmModel.addTermGenerator(crm.CRMSecElTermGenerator("secElCRM",f,secElInds))

rk.models.add(crmModel)


### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = rmk.BDEIntegrator("BDE",nonlinTol=1e-14,convergenceVars=[f,neutralVars[0],neutralVars[1],We,Te])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models)
rk.integrationScheme = rmk.IntegrationScheme(dt=0.5,steps=integrationStep)
rk.integrationScheme.setFixedNumTimesteps(30000,1000)


#### Generate LaTeX summary PDF

In [ ]:
rk.generatePDF("CRM 0D kinetic Example")

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

In [ ]:
eTot = dataset["We"].data[:,0] + dataset["ni"].data[:,0]*13.6/rk.norms["eVTemperature"]

for i,state in enumerate(neutralDensList):
    eTot = eTot + dataset[state].data[:,0] * 13.6 * (1 - 1/(i+1)**2)/rk.norms["eVTemperature"]

dE=abs(eTot - eTot[0])/eTot[0]

ntot = dataset["ni"].data[:,0]

for state in neutralDensList:
    ntot = ntot + dataset[state].data[:,0]

dn =abs(ntot-ntot[0])/ntot[0]

timeNorm = rk.norms["time"]

eCurve = hv.Curve([(t*timeNorm*1000,dE[i]) for i,t in enumerate(dataset.coords["t"])],label="E").opts(linestyle="--")

nCurve = hv.Curve([(t*timeNorm*1000,dn[i]) for i,t in enumerate(dataset.coords["t"])],label="n")

curve = nCurve*eCurve
curve.opts(xlabel="t [ms]",ylabel='$\delta$')


In [ ]:
hv.save(curve.opts(xlabel="t [ms]",ylabel='$\delta$'),"kin_crm_test.pdf",dpi=144)